<a href="https://colab.research.google.com/github/asifahsaan/Machine-Translation-NLP/blob/master/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM
from tensorflow.keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("eng_-french.csv")
data

,English words/sentences,French words/sentences
0,Hi.,Salut!
1,Run!,Cours !
2,Run!,Courez !
3,Who?,Qui ?
4,Wow!,Ça alors !
...,...,...
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç..."
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...


In [ ]:
data['english_sentences'] = data['English words/sentences'].str.split().str.len()
data['french_sentences'] = data['French words/sentences'].str.split().str.len()

In [ ]:
data

,English words/sentences,French words/sentences,english_sentences,french_sentences
0,Hi.,Salut!,1,1
1,Run!,Cours !,1,2
2,Run!,Courez !,1,2
3,Who?,Qui ?,1,2
4,Wow!,Ça alors !,1,3
...,...,...,...,...
175616,"Top-down economics never works, said Obama. ""T...","« L'économie en partant du haut vers le bas, ç...",34,47
175617,A carbon footprint is the amount of carbon dio...,Une empreinte carbone est la somme de pollutio...,34,33
175618,Death is something that we're often discourage...,La mort est une chose qu'on nous décourage sou...,37,47
175619,Since there are usually multiple websites on a...,Puisqu'il y a de multiples sites web sur chaqu...,43,49


In [ ]:
data = data.drop_duplicates()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [ ]:
print(data['english_sentences'].max(), data['french_sentences'].max())

44 55


In [ ]:
train, valid = train_test_split(train, test_size=0.2)

In [ ]:
def get_data(raw_lines):
  text=[]
  for raw_line in raw_lines:
    text.append('<start>' + raw_line + '<end>')
  return text

In [ ]:
eng_train = get_data(list(train['English words/sentences']))

In [ ]:
fre_train = get_data(list(train['French words/sentences']))

In [ ]:
eng_valid = get_data(list(train['English words/sentences']))
fre_valid = get_data(list(train['French words/sentences']))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#tokenizing and padding input data
fre_token = Tokenizer(filters = '', lower = False)
fre_token.fit_on_texts(fre_train)
fre_tokenized = fre_token.texts_to_sequences(fre_train)
fre_padded = pad_sequences(fre_tokenized, padding='post')

#tokenizing and padding target data
eng_token = Tokenizer(filters = '', lower = False)
eng_token.fit_on_texts(eng_train)
eng_tokenized = eng_token.texts_to_sequences(eng_train)
eng_padded = pad_sequences(fre_tokenized, padding='post')

In [ ]:
#number of unique tokens in input and output languages
num_opTokens = len(fre_token.word_index)
num_ipTokens = len(eng_token.word_index)

#max length of a sentence in both lang
max_len_op = fre_padded.shape[1]
max_len_ip = eng_padded.shape[1]

In [ ]:
fre_train[0]

'<start>Tom a quitté sa voiture.<end>'

In [ ]:
#num of unique tokens in both language
print(num_opTokens, num_ipTokens)

40287 23885


In [ ]:
#max length of sentence in both language
print(max_len_op, max_len_ip)

55 55


In [ ]:
!pip install ktext

In [ ]:
!pip install joblib

In [ ]:
!pip install --upgrade keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import joblib

In [ ]:
from ktext.preprocess import processor

In [ ]:
eng_pp = processor(keep_n=25407, padding_maxlen=55)
eng_train_vec = eng_pp.fit_transform(eng_train)

In [ ]:
fre_pp = processor(keep_n=41000, padding_maxlen=55, append_indicators=True, padding='post')
fre_train_vec = fre_pp.fit_transform(fre_train)

In [ ]:
import dill as dpickle
import numpy as np

with open('eng_pp.dpkl', 'wb') as f:
  dpickle.dump(eng_pp, f)

with open('fre_pp.dpkl', 'wb') as f:
  dpickle.dump(fre_pp, f)

np.save('eng_train_vecs.npy', eng_train_vec)
np.save('fre_train_vecs.npy', fre_train_vec)

In [ ]:
def load_encoder_input(encoder_np_vecs='eng_train_vecs.npy'):
  vectorizedBody = np.load(encoder_np_vecs)
  encoder_input_data = vectorizedBody
  doc_length  = encoder_input_data.shape[1]

  print(f'Shape of encoder input: {encoder_input_data.shape}')
  return encoder_input_data, doc_length

def load_decoder_input(decoder_np_vecs='fre_train_vecs.npy'):
  vectorizedTitle = np.load(decoder_np_vecs)
  decoder_input_data = vectorizedTitle[:, :-1]
  decoder_target_data = vectorizedTitle[:, :-1]

  print(f'Shape of decoder input: {decoder_input_data.shape}')
  print(f'Shape of decoder target: {decoder_target_data.shape}')
  return decoder_input_data, decoder_target_data

In [ ]:
def load_text_processor(fname='eng_pp.dpkl'):
  with open(fname, 'rb') as f:
    pp = dpickle.load(f)

    numTokens  = max(pp.id2token.keys()) + 1
    print(f'Size of vocabulary for {fname} : {numTokens:,}')
    return numTokens, pp

In [ ]:
num_encoder_tokens, eng_pp = load_text_processor('eng_pp.dpkl')
num_decoder_tokens, fre_pp = load_text_processor('fre_pp.dpkl')

Size of vocabulary for eng_pp.dpkl : 12,089
Size of vocabulary for fre_pp.dpkl : 19,409


In [ ]:
encoder_input_data, doc_length = load_encoder_input('eng_train_vecs.npy')
decoder_input_data, decoder_target_data = load_decoder_input('fre_train_vecs.npy')

Shape of encoder input: (112396, 55)
Shape of decoder input: (112396, 54)
Shape of decoder target: (112396, 54)


In [ ]:
print('\noriginal string:\n', eng_train[0], '\n')
print('\nAfter preprocessing:\n', encoder_input_data[0], '\n')


original string:
 <start>Tom left in his car.<end> 


After preprocessing:
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   2  12 180  19  44 128
   3] 



In [ ]:
print('\noriginal string:\n', fre_train[0], '\n')
print('After preprocessing:\n', decoder_input_data[0], '\n')


original string:
 <start>Tom a quitté sa voiture.<end> 

After preprocessing:
 [  3   4  21   7 831 102 138   2   5   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 



In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional, BatchNormalization
from keras import optimizers

In [ ]:
latent_dim = 300

encoder_inputs = Input(shape=(doc_length,), name='Encoder_Input')

x = Embedding(num_encoder_tokens, latent_dim, name='Body-Word_Embedding', mask_zero=False)(encoder_inputs)
x = BatchNormalization(name='Encoder-Batchnorm-1')(x)

_, state_h = GRU(latent_dim, return_state=True, name='Encoder-Last-GRU')(x)

encoder_model = Model(inputs=encoder_inputs, outputs=state_h, name='Encoder-Model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

decoder_inputs = Input(shape=(None,), name='Decoder-Input')
dec_emb = Embedding(num_decoder_tokens, latent_dim, name='Decoder-Word-Embedding', mask_zero=False)(decoder_inputs)
dec_bn = BatchNormalization(name='Decoder-Batchnorm-1')(dec_emb)

decoder_gru  = GRU(latent_dim, return_state=True, return_sequences=True, name='Decoder-GRU')
decoder_gru_output, _ = decoder_gru(dec_bn, initial_state=seq2seq_encoder_out)
x = BatchNormalization(name='Decoder-Batchnorm-2')(decoder_gru_output)

decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Final-Output-Dense')
decoder_outputs = decoder_dense(x)



seq2seq_Model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
seq2seq_Model.compile(optimizer=optimizers.Nadam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

In [ ]:
seq2seq_Model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Decoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Decoder-Word-Embedding (Embedd  (None, None, 300)   5822700     ['Decoder-Input[0][0]']          
 ing)                                                                                             
                                                                                                  
 Encoder_Input (InputLayer)     [(None, 55)]         0           []                               
                                                                                                  
 Decoder-Batchnorm-1 (BatchNorm  (None, None, 300)   1200        ['Decoder-Word-Embedding[0][0

In [ ]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping

script_name_base = 'MT_seq2seq'
csv_logger = CSVLogger('{:}.log'.format(script_name_base))
model_checkpoint = ModelCheckpoint('{:}.epoch{{epoch:02d}}--val{{val_loss:.5f}}.hdf5'.format(script_name_base), 
                                   save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)

batch_size = 1200
epochs = 10
history = seq2seq_Model.fit([encoder_input_data, decoder_input_data], np.expand_dims(decoder_target_data, axis=-1),
                            batch_size=batch_size, epochs=epochs, validation_split=0.12,
                            callbacks=[csv_logger, model_checkpoint, early_stopping])

Epoch 1/10
83/83 [==============================] - 127s 1s/step - loss: 2.6370 - val_loss: 8.2670
Epoch 2/10
83/83 [==============================] - 123s 1s/step - loss: 0.0825 - val_loss: 8.4812
Epoch 3/10
83/83 [==============================] - 120s 1s/step - loss: 0.0280 - val_loss: 7.7384
Epoch 4/10
83/83 [==============================] - 119s 1s/step - loss: 0.0119 - val_loss: 6.3780
Epoch 5/10
83/83 [==============================] - 124s 1s/step - loss: 0.0047 - val_loss: 4.3411
Epoch 6/10
83/83 [==============================] - 120s 1s/step - loss: 0.0026 - val_loss: 1.8466
Epoch 7/10
83/83 [==============================] - 119s 1s/step - loss: 0.0019 - val_loss: 0.3769
Epoch 8/10
83/83 [==============================] - 119s 1s/step - loss: 0.0015 - val_loss: 0.0546
Epoch 9/10
83/83 [==============================] - 119s 1s/step - loss: 0.0012 - val_loss: 0.0213
Epoch 10/10
83/83 [==============================] - 119s 1s/step - loss: 0.0010 - val_loss: 0.0169


In [ ]:
del seq2seq_Model

In [ ]:
seq2seq_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
seq2seq_Model.compile(optimizer=optimizers.Nadam(learning_rate=0.001), loss='sparse_categorical_crossentropy')

In [ ]:
seq2seq_Model.load_weights('MT_seq2seq.epoch10--val0.01690.hdf5')

In [ ]:
def extract_encoder_model(model):
  encoder_model = model.get_layer('Encoder-Model')
  return encoder_model

def extract_decoder_model(model):
  latent_dim = model.get_layer('Decoder-Word-Embedding').output_shape[-1]

  decoder_inputs = model.get_layer('Decoder-Input').input
  dec_emb = model.get_layer('Decoder-Word-Embedding')(decoder_inputs)
  dec_bn = model.get_layer('Decoder-Batchnorm-1')(dec_emb)

  gru_inference_state_input = Input(shape=(latent_dim,), name='hidden_state_input')

  gru_out, gru_state_out = model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input])

  dec_bn2 = model.get_layer('Decoder-Batchnorm-2')(gru_out)
  dense_out = model.get_layer('Final-Output-Dense')(dec_bn2)
  decoder_model = Model([decoder_inputs, gru_inference_state_input],
                        [dense_out, gru_state_out])
  return decoder_model